# Data preparation

We assume that the data structure is the same as in the 'sample.csv'. What we want is to extract the following information for each row: ['time','id','lat','long'].
Here, 
- id is ip_address (we removed '.' from them; we can use another unique identifier for ex device id, but this is not currently implemented)
- time is bid_time
- lat and long are geo_lat and geo_long

This 4 information per event is enough for our other algorithms.

In [3]:
import pandas as pd
df = pd.read_csv('sample.csv')
df.head(5)

,ad_position,app_bundle,app_id,app_name,auction_id,bid_time,category,content_coppa_flag,content_language,content_rating,...,video_protocols,banner_top_frame,user_time_of_week,bid_time_utc,inventory_source_user_id,mccmnc,us_privacy,video_placement,experiment_user_index,publisher_name
0,NaN,484115113,sas/240574,Leboncoin,1585556136186356.2004355288.28624.admoove,2020-03-30 04:15:36.186,NaN,NaN,NaN,NaN,...,NaN,NaN,2055,2020-03-30 08:15:36.186,1609249544,NaN,NaN,NaN,294,NaN
1,NaN,454988262,sas/123389,Tl-Loisirs,1585556135722256.3093335463.28769.admoove,2020-03-30 04:15:35.722,"IAB1,IAB12,IAB19,IAB9",NaN,NaN,NaN,...,NaN,NaN,2055,2020-03-30 08:15:35.722,NaN,NaN,NaN,NaN,47,NaN
2,NaN,com.backelite.vingtminutes,sas/78802,20 Minutes,1585556136233584.3629637216.28605.admoove,2020-03-30 04:15:36.233,IAB12,NaN,NaN,NaN,...,NaN,NaN,2055,2020-03-30 08:15:36.233,1368056416,NaN,NaN,NaN,676,NaN
3,NaN,484115113,sas/240574,Leboncoin,1585556136228517.2141154957.28621.admoove,2020-03-30 04:15:36.228,NaN,NaN,NaN,NaN,...,NaN,NaN,2055,2020-03-30 08:15:36.228,1029488843,NaN,NaN,NaN,628,NaN
4,NaN,359917414,om/ce70e6a401257f3b113747cc91b2abc9,Solitaire,1585556136404412.2629489717.28636.admoove,2020-03-30 04:15:36.404,IAB9,NaN,NaN,NaN,...,NaN,NaN,2055,2020-03-30 08:15:36.404,a12b7d4e-be09-5fce-9a1a-1a1690bc2091,NaN,NaN,NaN,365,NaN


## Here is the data cleaning function 

The following function clean all the unwanted columns, and remove rows with null latitude and longitude (from command line you can .

In [24]:
import string 
from utils import remove_punctuation

# You can use this function by passing a Pandas Dataframe to it.
def data_prepare(df, cleaned_file):

    # drop unnecessary columns
    df = df.drop(['ad_position', 'app_bundle', 'app_id', 'app_name', 'auction_id', 'category', 'content_coppa_flag',
                  'content_language','content_rating', 'domain', 'environment_type', 'inventory_interstitial', 
                  'inventory_source','inventory_source_relationship', 'placement', 'platform_bandwidth', 
                  'platform_browser', 'platform_browser_version', 'platform_carrier', 'platform_device_didmd5', 
                  'platform_device_didsha1', 'platform_device_dpidmd5', 'platform_device_dpidsha1',
                  'platform_device_idfa', 'platform_device_ifa', 'platform_device_make', 'platform_device_model', 
                  'platform_device_screen_size', 'platform_device_type', 'platform_js', 'platform_os',
                  'platform_os_version', 'segment_id', 'segment_user_id', 'site_id', 'site_name','video_boxing_allowed', 
                  'video_companion_required', 'video_playback_method', 'video_player_size', 'video_start_delay', 'test', 
                  'placement_type','bid_time_epoch_in_usecs', 'page_url', 'exchange_predicted_view_rate', 
                  'available_deal_ids', 'exchange_auction_id', 'rewarded', 'bid_floor_micros', 'bid_floor_currency', 
                  'display_manager', 'display_manager_ver', 'exchange_device_make', 'exchange_device_model', 'user_id_type', 
                  'auction_type', 'publisher_id', 'ads_txt', 'matched_user_groups','video_protocols', 'banner_top_frame',
                  'inventory_source_user_id', 'mccmnc', 'us_privacy', 'video_placement', 'experiment_user_index', 
                  'publisher_name'], axis = 1 )

    # drop nan columns
    df = df.dropna(axis = 1)

    # let us drop further 'geo_type', 'is_gdpr', 'request_id', 'geo_country' since they are constant (at least in this sample)
    # let us also drop bid_time_utc (same as 'bid_time') and ip_range (same as ip_address)
    # no need fo time of week or usertime of week
    df =  df.drop(['geo_type', 'is_gdpr', 'request_id', 'geo_country','bid_time_utc','ip_range', 
                  'time_of_week','user_time_of_week'], axis=1)

    #print (df.columns)
    df = df[df['geo_lat'] != 0] # drop rows with zero latitude 
    df = df[df['geo_long'] != 0] # drop rows with zero latitude 


    exclude = set(string.punctuation) 
    df['ip_address'] = df['ip_address'].apply(lambda ch: remove_punctuation(ch))
    df['bid_time'] = pd.to_datetime(df['bid_time'])

    df.columns = ['time','id','lat','long']

    df.to_csv(cleaned_file,  index=False)
    
    return 

For consistency, it is best if 'data_prepare' function is applied day by day, and the cleaned_file the name of the corresponding day. What I mean by that: you should loop through your 'daily' files, read each day into a dataframe df and pass the corresponding date (for example, "03.01.2020") as argument: data_prepare(df, "03.01.2020.csv". At least this is how I imagined it would be.

With the sample file (sample.csv), this would give the following:

In [25]:
filename = 'sample.csv' #change path of files 
df = pd.read_csv(filename)
data_prepare(df,'03.01.2020.csv')

For sake of illustration, the output is in the following format:

In [26]:
output = pd.read_csv('03.01.2020.csv')
output.head(5)

,time,id,lat,long
0,2020-03-30 04:15:36.186,90.59.53.79,48.186268,0.652216
1,2020-03-30 04:15:35.722,83.193.39.0,43.506000,-1.477000
2,2020-03-30 04:15:36.233,176.145.161.215,43.693695,5.033779
3,2020-03-30 04:15:36.228,83.199.173.224,48.730705,2.587640
4,2020-03-30 04:15:36.404,77.147.33.204,44.915400,-0.427000
